# Полный анализ FIFA 2021: очистка, исследование и статистика

Этот ноутбук объединяет все этапы анализа данных по игрокам FIFA 2021:
- **Очистка данных**: Загрузка, приведение в порядок названий столбцов, преобразование типов, обработка пропусков и дубликатов, создание новых признаков.
- **Исследовательский анализ**: Фильтрация, сортировка, группировка данных и создание сводных таблиц для выявления ключевых закономерностей.
- **Статистический анализ**: Расчет описательных статистик, корреляционный анализ, проверка гипотез и визуализация данных.

## 1. Очистка данных (data_cleaning.py)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

try:
    df = pd.read_csv('players.csv', encoding='utf-8')
    print(f'Данные успешно загружены. Исходный размер: {df.shape}')
except Exception as e:
    print(f'Ошибка при загрузке файла: {e}')

### 1.1. Преобразование названий столбцов и текстовых данных

In [ ]:
df.columns = (df.columns
              .str.lower()
              .str.replace(' ', '_')
              .str.replace('-', '_')
              .str.replace('&', 'and')
              .str.replace('↓', '')
              .str.replace('/', '_')
              .str.replace('(', '')
              .str.replace(')', '')
              .str.strip('_'))

print("Названия столбцов после приведения к змеиному регистру:")
print(df.columns.tolist())

In [ ]:
text_columns = df.select_dtypes(include=['object']).columns
for col in text_columns:
    if col not in ['id', 'playerurl', 'photourl', 'name', 'full_name']:
        df[col] = df[col].astype(str).str.lower().str.strip()

print("Текстовые данные приведены к нижнему регистру (кроме имен и ссылок).")

### 1.2. Создание новых признаков и преобразование данных

In [ ]:
if 'team_and_contract' in df.columns:
    def split_team_contract(value):
        if pd.isna(value) or value == 'nan':
            return pd.Series([np.nan, np.nan, np.nan])
        value = str(value).strip()
        lines = value.split('\n')
        if len(lines) >= 2:
            team = lines[0].strip()
            contract = lines[1].strip()
            if '~' in contract:
                years = contract.split('~')
                if len(years) == 2:
                    try:
                        start_year = int(years[0].strip())
                        end_year = int(years[1].strip())
                    except ValueError:
                        start_year = np.nan
                        end_year = np.nan
                    return pd.Series([team, start_year, end_year])
            return pd.Series([team, np.nan, np.nan])
        else:
            return pd.Series([value, np.nan, np.nan])
    
    df[['club_name', 'contract_start_year', 'contract_end_year']] = df['team_and_contract'].apply(split_team_contract)
    df = df.drop('team_and_contract', axis=1)
    print('Столбец team_and_contract разделен на club_name, contract_start_year, contract_end_year')

In [ ]:
if 'positions' in df.columns:
    positions_split = df['positions'].astype(str).str.split()
    all_positions = set()
    for positions in positions_split.dropna():
        if isinstance(positions, list):
            all_positions.update(positions)
    all_positions.discard('nan')
    for position in sorted(all_positions):
        df[f'position_{position.lower()}'] = positions_split.apply(
            lambda x: 1 if isinstance(x, list) and position in x else 0
        )
    df['positions_formatted'] = df['positions'].astype(str).str.replace(' ', ',')
    print('Позиции преобразованы: созданы dummy-переменные и столбец positions_formatted.')

In [ ]:
if 'height' in df.columns:
    def convert_height(height_str):
        if pd.isna(height_str):
            return np.nan
        height_str = str(height_str).strip()
        if "'" in height_str:
            parts = height_str.replace('"', '').split("'")
            if len(parts) == 2:
                feet = int(parts[0])
                inches = int(parts[1]) if parts[1] else 0
                return feet * 30.48 + inches * 2.54
        return np.nan
    df['height_cm'] = df['height'].apply(convert_height)
    df = df.drop('height', axis=1)
    print('Рост преобразован из футов и дюймов в сантиметры.')

In [ ]:
if 'weight' in df.columns:
    df['weight_kg'] = df['weight'].astype(str).str.extract('(\d+)').astype(float) * 0.453592
    df = df.drop('weight', axis=1)
    print('Вес преобразован из фунтов в килограммы.')

In [ ]:
star_columns = [col for col in df.columns if '★' in str(col) or 'star' in str(col).lower()]
for col in star_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.extract('(\d+)').astype(float)

columns_to_clean_stars = ['w_f', 'ir', 'sm']
for col in columns_to_clean_stars:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('★', '', regex=False).str.extract('(\d+)').astype(float)
print('Звездные рейтинги (★) преобразованы в числовой формат.')

### 1.3. Обработка пропущенных значений и дубликатов

In [ ]:
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_info = pd.DataFrame({'количество_пропусков': missing_data, 'процент_пропусков': missing_percentage})
missing_info = missing_info[missing_info['количество_пропусков'] > 0]
print('Пропуски в данных до обработки:')
print(missing_info)

if not missing_info.empty:
    print("\nОбработка пропущенных значений...")
    for col in missing_info.index:
        missing_pct = missing_info.loc[col, 'процент_пропусков']
        if missing_pct <= 5:
            if df[col].dtype in ['int64', 'float64', 'float32']:
                if abs(df[col].skew()) > 1:
                    fill_value = df[col].median()
                    df[col].fillna(fill_value, inplace=True)
                else:
                    fill_value = df[col].mean()
                    df[col].fillna(fill_value, inplace=True)
            else:
                fill_value = df[col].mode()[0] if not df[col].mode().empty else 'unknown'
                df[col].fillna(fill_value, inplace=True)
        else:
            print(f"Столбец {col} имеет {missing_pct:.1f}% пропусков, требует индивидуального рассмотрения.")

In [ ]:
initial_shape = df.shape
duplicates = df.duplicated()
duplicate_count = duplicates.sum()
print(f"\nКоличество дубликатов: {duplicate_count}")

if duplicate_count > 0:
    df = df.drop_duplicates()
    print(f"Дубликаты удалены. Размер после удаления: {df.shape}")
else:
    print("Дубликатов не найдено.")

### 1.4. Оптимизация типов данных и сохранение

In [ ]:
for col in df.columns:
    old_type = df[col].dtype
    if df[col].dtype == 'int64':
        if df[col].min() >= 0 and df[col].max() <= 255:
            df[col] = df[col].astype('uint8')
    elif df[col].dtype == 'float64':
        if df[col].min() >= np.finfo(np.float32).min and df[col].max() <= np.finfo(np.float32).max:
            df[col] = df[col].astype('float32')
    new_type = df[col].dtype
    if old_type != new_type:
        print(f"Столбец '{col}' оптимизирован: {old_type} -> {new_type}")

columns_to_drop = ['value', 'wage', 'release_clause']
df = df.drop(columns=columns_to_drop, errors='ignore')

df.to_csv('players_cleaned.csv', index=False)
print(f"\nОчищенные данные сохранены в 'players_cleaned.csv'. Финальный размер: {df.shape}")

## 2. Исследовательский анализ (EDA) (analiz-2.py)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

try:
    df = pd.read_csv('players_cleaned.csv')
    print(f'Очищенные данные загружены. Размер датасета: {df.shape}')
except FileNotFoundError:
    print('Файл players_cleaned.csv не найден. Сначала выполните очистку данных.')
    df = None

### 2.1. Индексация и фильтрация данных

In [ ]:
if df is not None:
    print("1. Игроки с общим рейтингом выше 90:")
    high_rated = df[df['ova'] > 90]
    print(high_rated[['longname', 'age', 'ova', 'pot', 'nationality']].head())

    print("\n2. Молодые игроки (до 20 лет) с потенциалом выше 85:")
    young_talents = df.query('age < 20 and pot > 85')
    print(young_talents[['longname', 'age', 'ova', 'pot', 'nationality']].head())

### 2.2. Сортировка данных

In [ ]:
if df is not None:
    print("Топ-10 игроков по общему рейтингу:")
    top_rated = df.sort_values('ova', ascending=False).head(10)
    print(top_rated[['longname', 'age', 'ova', 'pot', 'nationality']])

    print("\nТоп-10 молодых игроков по потенциалу:")
    young_potential = df.sort_values('pot', ascending=False).head(10)
    print(young_potential[['longname', 'age', 'ova', 'pot', 'nationality']])

### 2.3. Сводные таблицы и группировка

In [ ]:
if df is not None:
    print("Сводная таблица 1: Средний рейтинг по топ-10 национальностям:")
    pivot1 = df.pivot_table(values='ova', index='nationality', aggfunc=['mean', 'count'])
    pivot1.columns = ['Средний рейтинг', 'Количество игроков']
    print(pivot1.sort_values('Средний рейтинг', ascending=False).head(10).round(2))

    # Создаем категорию позиции для группировки
    def categorize_position(pos):
        if 'gk' in pos.lower(): return 'Вратарь'
        elif any(x in pos.lower() for x in ['cb', 'lb', 'rb', 'wb']): return 'Защитник'
        elif any(x in pos.lower() for x in ['cm', 'cdm', 'cam']): return 'Полузащитник'
        elif any(x in pos.lower() for x in ['st', 'cf', 'lw', 'rw']): return 'Нападающий'
        else: return 'Другое'
    df['position_category'] = df['positions_formatted'].apply(categorize_position)

    print("\nГруппировка по категориям позиций:")
    position_stats = df.groupby('position_category').agg(
        {'ova': 'mean', 'height_cm': 'mean', 'weight_kg': 'mean', 'sprint_speed': 'mean'}
    ).round(2)
    print(position_stats)

### 2.4. Анализ потенциального роста

In [ ]:
if df is not None:
    df['potential_growth'] = df['pot'] - df['ova']
    print(f"Средний рост потенциала: {df['potential_growth'].mean():.2f}")
    print("Игроки с наибольшим потенциалом роста:")
    growth_players = df.nlargest(10, 'potential_growth')
    print(growth_players[['longname', 'age', 'ova', 'pot', 'nationality', 'potential_growth']])

## 3. Статистический анализ (statistical_analysis.py)

### 3.1. Основные статистические показатели

In [ ]:
if df is not None:
    key_numeric_columns = [
        'age', 'ova', 'pot', 'height_cm', 'weight_kg', 'sprint_speed', 
        'dribbling', 'shot_power', 'short_passing', 'stamina', 'reactions'
    ]
    key_numeric_columns = [col for col in key_numeric_columns if col in df.columns]
    print("Основные статистические показатели по ключевым признакам:")
    print(df[key_numeric_columns].describe().round(2))

### 3.2. Корреляционный анализ

In [ ]:
if df is not None:
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    corr_matrix = df[numeric_cols].corr()
    
    print("Топ-10 корреляций с общим рейтингом (ova):")
    ova_correlations = corr_matrix['ova'].sort_values(ascending=False)
    print(ova_correlations.head(11).round(3))
    
    plt.figure(figsize=(12, 9))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
    plt.title('Корреляционная матрица числовых признаков')
    plt.show()

### 3.3. Визуализация распределений

In [ ]:
if df is not None:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Распределения ключевых числовых признаков', fontsize=16)

    key_features = ['age', 'ova', 'height_cm', 'sprint_speed', 'dribbling', 'stamina']
    titles = ['Возраст', 'Общий рейтинг', 'Рост (см)', 'Скорость', 'Дриблинг', 'Выносливость']

    for i, (feature, title) in enumerate(zip(key_features, titles)):
        row, col = i // 3, i % 3
        sns.histplot(df[feature], bins=30, kde=True, ax=axes[row, col], color='skyblue')
        axes[row, col].set_title(f'Распределение: {title}')
        axes[row, col].axvline(df[feature].mean(), color='red', linestyle='--', label=f'Среднее: {df[feature].mean():.1f}')
        axes[row, col].legend()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

### 3.4. Проверка статистических гипотез

In [ ]:
from scipy.stats import ttest_ind, chi2_contingency

if df is not None:
    # Гипотеза 1: Средний рейтинг молодых игроков (до 25 лет) отличается от опытных (25+)
    young_players = df[df['age'] < 25]['ova']
    experienced_players = df[df['age'] >= 25]['ova']
    t_stat, p_value = ttest_ind(young_players, experienced_players)
    print(f"Т-тест (рейтинг молодых vs опытных): t-статистика = {t_stat:.2f}, p-значение = {p_value:.5f}")
    if p_value < 0.05:
        print("Результат: Отклоняем нулевую гипотезу. Средние рейтинги статистически значимо различаются.")
    else:
        print("Результат: Не отклоняем нулевую гипотезу.")

    # Гипотеза 2: Возраст и рейтинг являются зависимыми переменными
    df['age_group'] = pd.cut(df['age'], bins=[0, 20, 25, 30, 35, 100], labels=['До 20', '20-25', '25-30', '30-35', '35+'])
    df['rating_group'] = pd.cut(df['ova'], bins=[0, 60, 70, 80, 85, 100], labels=['До 60', '60-70', '70-80', '80-85', '85+'])
    contingency_table = pd.crosstab(df['age_group'], df['rating_group'])
    chi2, p_val, dof, expected = chi2_contingency(contingency_table)
    print(f"\nХи-квадрат тест (возраст и рейтинг): chi2 = {chi2:.2f}, p-значение = {p_val:.5f}")
    if p_val < 0.05:
        print("Результат: Отклоняем нулевую гипотезу. Возрастные группы и группы рейтинга зависимы.")
    else:
        print("Результат: Не отклоняем нулевую гипотезу.")